<a href="https://colab.research.google.com/github/qowogks123/AISPARK_challenge/blob/main/AI_SPARK_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌**2023 연구개발특구 AI SPARK 챌린지 - 사회문제해결형** 
##**부제 : (초미세먼지(PM2.5) 오염도 단기 예측 모델 만들기)**
   
### **팀원: 배재한, 김정현, 김민수**

### **1. 구글 드라이브 마운트**

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **2. 필요 라이브러리 및 모듈 호출**



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### **3-1 데이터 프레임 불러오기**

1.   학습 데이터
2.   기상정보가 포함된 AWS 데이터

3. 결측치 확인

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/data/train.csv')
train_df

,연도,일시,측정소,PM2.5
0,0,01-01 00:00,모종동,NaN
1,0,01-01 01:00,모종동,NaN
2,0,01-01 02:00,모종동,NaN
3,0,01-01 03:00,모종동,NaN
4,0,01-01 04:00,모종동,NaN
...,...,...,...,...
596083,3,12-31 19:00,홍성읍,0.060
596084,3,12-31 20:00,홍성읍,0.052
596085,3,12-31 21:00,홍성읍,0.044
596086,3,12-31 22:00,홍성읍,0.052


In [ ]:
aws_df = pd.read_csv('/content/drive/MyDrive/data/aws.csv')
aws_df

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,계룡,0.154818,0.410833,0.033248,0.0,0.830
1,0,01-01 01:00,계룡,0.142180,0.148611,0.030691,0.0,0.853
2,0,01-01 02:00,계룡,0.173776,0.203889,0.040921,0.0,0.802
3,0,01-01 03:00,계룡,0.169036,0.330556,0.048593,0.0,0.816
4,0,01-01 04:00,계룡,0.169036,0.000000,0.007673,0.0,0.827
...,...,...,...,...,...,...,...,...
1051915,3,12-31 19:00,홍성죽도,0.317536,0.953333,0.230179,0.0,0.630
1051916,3,12-31 20:00,홍성죽도,0.309637,0.984167,0.253197,0.0,0.648
1051917,3,12-31 21:00,홍성죽도,0.308057,0.966667,0.265985,0.0,0.648
1051918,3,12-31 22:00,홍성죽도,0.306477,0.976667,0.235294,0.0,0.642


In [ ]:
missing_values = train_df.isnull().sum()

In [ ]:
# 초미세먼지 - PM2.5에서 15542의 결측 확인
missing_values

연도           0
일시           0
측정소          0
PM2.5    15542
dtype: int64

### **3-2 데이터 전처리**

1. PM2.5 측정소 위치와 AWS(기상정보) 측정 지점의 위치가 분산되어있으므로 인근 지점으로 매핑 
2. 연도,일시,측정소 컬럼 기준으로 병합

In [ ]:
# 30지점 16측정소
aws_df.loc[aws_df['지점'] == "계룡", "지점"] = "노은동"
aws_df.loc[aws_df['지점'] == "세천", "지점"] = "문창동"
aws_df.loc[aws_df['지점'] == "오월드", "지점"] = "정림동"
aws_df.loc[aws_df['지점'] == "장동", "지점"] = "읍내동"
aws_df.loc[aws_df['지점'] == "세종금남", "지점"] = "아름동"
aws_df.loc[aws_df['지점'] == "세종고운", "지점"] = "아름동"
aws_df.loc[aws_df['지점'] == "공주", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "정산", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "유구", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "정안", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "청양", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "대천항", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "호도", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "춘장대", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "서천", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "논산", "지점"] = "논산"
aws_df.loc[aws_df['지점'] == "양화", "지점"] = "논산"
aws_df.loc[aws_df['지점'] == "홍북", "지점"] = "홍성읍"
aws_df.loc[aws_df['지점'] == "홍성죽도", "지점"] = "홍성읍"
aws_df.loc[aws_df['지점'] == "옹도", "지점"] = "이원면"
aws_df.loc[aws_df['지점'] == "안도", "지점"] = "이원면"
aws_df.loc[aws_df['지점'] == "북격렬비도", "지점"] = "이원면"
aws_df.loc[aws_df['지점'] == "태안", "지점"] = "동문동"
aws_df.loc[aws_df['지점'] == "당진", "지점"] = "동문동"
aws_df.loc[aws_df['지점'] == "대산", "지점"] = "독곶리"
aws_df.loc[aws_df['지점'] == "예산", "지점"] = "예산군"
aws_df.loc[aws_df['지점'] == "아산", "지점"] = "모종동"
aws_df.loc[aws_df['지점'] == "성거", "지점"] = "성성동"
aws_df.loc[aws_df['지점'] == "세종전의", "지점"] = "신방동"
aws_df.loc[aws_df['지점'] == "세종연서", "지점"] = "신흥동"

In [ ]:
aws_df.rename(columns = {"지점":"측정소"},inplace=True)
aws_df

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830
1,0,01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853
2,0,01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802
3,0,01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816
4,0,01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827
...,...,...,...,...,...,...,...,...
1051915,3,12-31 19:00,홍성읍,0.317536,0.953333,0.230179,0.0,0.630
1051916,3,12-31 20:00,홍성읍,0.309637,0.984167,0.253197,0.0,0.648
1051917,3,12-31 21:00,홍성읍,0.308057,0.966667,0.265985,0.0,0.648
1051918,3,12-31 22:00,홍성읍,0.306477,0.976667,0.235294,0.0,0.642


In [ ]:
merged_df = pd.merge(aws_df, train_df, on=['연도','일시','측정소'])
merged_df

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0,01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0,01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0,01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0,01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...,...
1051915,3,12-31 21:00,홍성읍,0.308057,0.966667,0.265985,0.0,0.648,0.044
1051916,3,12-31 22:00,홍성읍,0.262243,0.866944,0.043478,0.0,0.725,0.052
1051917,3,12-31 22:00,홍성읍,0.306477,0.976667,0.235294,0.0,0.642,0.052
1051918,3,12-31 23:00,홍성읍,0.257504,0.000000,0.000000,0.0,0.710,0.060


In [ ]:
# 잘 병합되었는지 확인
x = merged_df.loc[merged_df["측정소"]=="모종동","PM2.5"]
x

771408      NaN
771409      NaN
771410      NaN
771411      NaN
771412      NaN
          ...  
806467    0.036
806468    0.048
806469    0.044
806470    0.048
806471    0.044
Name: PM2.5, Length: 35064, dtype: float64

In [ ]:
merged_df.측정소.unique()

array(['노은동', '이원면', '성성동', '동문동', '대천2동', '독곶리', '논산', '공주', '아름동',
       '예산군', '모종동', '신방동', '읍내동', '문창동', '정림동', '신흥동', '홍성읍'],
      dtype=object)

In [ ]:
aws_df.측정소.unique()

array(['노은동', '이원면', '성성동', '동문동', '대천2동', '독곶리', '논산', '공주', '아름동',
       '예산군', '모종동', '신방동', '읍내동', '문창동', '정림동', '신흥동', '홍성읍'],
      dtype=object)

In [ ]:
# 베이스라인 학습 데이터셋 구성
merged_df.to_csv('final_df.csv')

### **4. 시계열 데이터 가공**

1. [일시] 컬럼을 연-월-일 시:분으로 재정의 
  - Datetime 모듈 사용
  - pd.to_Datime 모듈 사용
2. label Encoder를 사용하여 범주형 변수 수치로 변환

In [ ]:
# 더미 데이터프레임으로 카피하여 사용
df = merged_df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051920 entries, 0 to 1051919
Data columns (total 9 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   연도       1051920 non-null  int64  
 1   일시       1051920 non-null  object 
 2   측정소      1051920 non-null  object 
 3   기온(°C)   1032972 non-null  float64
 4   풍향(deg)  1032972 non-null  float64
 5   풍속(m/s)  1032972 non-null  float64
 6   강수량(mm)  1032972 non-null  float64
 7   습도(%)    1032972 non-null  float64
 8   PM2.5    1023621 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 80.3+ MB


In [ ]:
# 결측치 확인
df.isna().sum()

연도             0
일시             0
측정소            0
기온(°C)     18948
풍향(deg)    18948
풍속(m/s)    18948
강수량(mm)    18948
습도(%)      18948
PM2.5      28299
dtype: int64

## Imputer 사용

In [ ]:
# 2020년에 2월29일이 존재함으로 임의로 년도 정의 및 매핑
df.loc[df['연도'] == 0, '일시'] = '2017-' + df.loc[df['연도'] == 0, '일시']
df.loc[df['연도'] == 1, '일시'] = '2018-' + df.loc[df['연도'] == 1, '일시']
df.loc[df['연도'] == 2, '일시'] = '2019-' + df.loc[df['연도'] == 2, '일시']
df.loc[df['연도'] == 3, '일시'] = '2020-' + df.loc[df['연도'] == 3, '일시']

In [ ]:
# Datetime 형식으로 변환
df['일시'] = df['일시'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M'))

In [ ]:
# 시계열 학습을 위해 ["일시"] 컬럼을 인덱스로 설정한다.
df = df.set_index(["일시"])

In [ ]:
# labelencoder를 사용하여 범주형 변수를 수치형으로 변환한다.
le = LabelEncoder()
df['측정소'] = le.fit_transform(df['측정소'])
df

,연도,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
일시,,,,,,,,
2017-01-01 00:00:00,0,1,0.154818,0.410833,0.033248,0.0,0.830,0.052
2017-01-01 01:00:00,0,1,0.142180,0.148611,0.030691,0.0,0.853,0.072
2017-01-01 02:00:00,0,1,0.173776,0.203889,0.040921,0.0,0.802,0.068
2017-01-01 03:00:00,0,1,0.169036,0.330556,0.048593,0.0,0.816,0.064
2017-01-01 04:00:00,0,1,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...
2020-12-31 21:00:00,3,16,0.308057,0.966667,0.265985,0.0,0.648,0.044
2020-12-31 22:00:00,3,16,0.262243,0.866944,0.043478,0.0,0.725,0.052
2020-12-31 22:00:00,3,16,0.306477,0.976667,0.235294,0.0,0.642,0.052


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

iterimp = IterativeImputer()
df= pd.DataFrame(iterimp.fit_transform(df), columns = df.columns)

In [ ]:
df.isna().sum()

연도         0
측정소        0
기온(°C)     0
풍향(deg)    0
풍속(m/s)    0
강수량(mm)    0
습도(%)      0
PM2.5      0
dtype: int64

In [ ]:
# 임시로 결측치를 0으로 채움
#df = df.fillna(0)

In [ ]:
# 데이터 확인
df

,연도,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0.0,1.0,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0.0,1.0,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0.0,1.0,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0.0,1.0,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0.0,1.0,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...
1051915,3.0,16.0,0.308057,0.966667,0.265985,0.0,0.648,0.044
1051916,3.0,16.0,0.262243,0.866944,0.043478,0.0,0.725,0.052
1051917,3.0,16.0,0.306477,0.976667,0.235294,0.0,0.642,0.052
1051918,3.0,16.0,0.257504,0.000000,0.000000,0.0,0.710,0.060


In [ ]:
df

,연도,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0.0,1.0,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0.0,1.0,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0.0,1.0,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0.0,1.0,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0.0,1.0,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...
1051915,3.0,16.0,0.308057,0.966667,0.265985,0.0,0.648,0.044
1051916,3.0,16.0,0.262243,0.866944,0.043478,0.0,0.725,0.052
1051917,3.0,16.0,0.306477,0.976667,0.235294,0.0,0.642,0.052
1051918,3.0,16.0,0.257504,0.000000,0.000000,0.0,0.710,0.060


In [ ]:
# 변환되었는지 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051920 entries, 0 to 1051919
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   연도       1051920 non-null  float64
 1   측정소      1051920 non-null  float64
 2   기온(°C)   1051920 non-null  float64
 3   풍향(deg)  1051920 non-null  float64
 4   풍속(m/s)  1051920 non-null  float64
 5   강수량(mm)  1051920 non-null  float64
 6   습도(%)    1051920 non-null  float64
 7   PM2.5    1051920 non-null  float64
dtypes: float64(8)
memory usage: 64.2 MB


## **5. AI 학습**

- 타깃 변수 지정과 x, y 설정
- train_test_split 사용으로 학습과 테스트데이터 분류

In [ ]:
target = "PM2.5"
y = df[target]
x = df.drop(columns = target)

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.model_selection import GridSearchCV

### 5-1. *Ensemble 모델 사용*

In [ ]:
#GridSearchCV 를 사용할 모델들을 호출합니다.
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,ExtraTreesRegressor


#모델들을 할당할 리스트를 만들어줍니다.
estimators = []

#estimators 리스트에 모델들을 추가해줍니다.
#model_rf = RandomForestRegressor()
#estimators.append(model_rf)

model_gbr = GradientBoostingRegressor()
estimators.append(model_gbr)

#model_etr = ExtraTreesRegressor()
#estimators.append(model_etr)

#모들의 파라미터들을 할당할 리스트를 만들어줍니다.
params = []

# params 리스트에 성능을 비교하고자하는 파라미터들 추가해줍니다.
#params_rf = {'n_estimators' : [50, 150],
#            'min_samples_split' : [1,3,4]}
#params.append(params_rf)

params_gbr = {'loss' : ['squared_error', 'absolute_error'],
             'learning_rate':[0.05,0.1],
             'n_estimators':[50,150]}
params.append(params_gbr)

#params_etr = {'n_estimators' : [50,150]}
#params.append(params_etr)

In [ ]:
from tqdm.auto import tqdm
def gridSearchCV(models,params):
    best_models=[]
    for i in tqdm(range(0,len(models))):
        model_grid = GridSearchCV(models[i], params[i], verbose=1, cv=5)
        model_grid.fit(x_train,y_train)
        best_models.append(model_grid.best_estimator_)
    return best_models

best_model_list = gridSearchCV(estimators,params)

  0%|          | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [ ]:
best_model_list

[GradientBoostingRegressor(n_estimators=150)]

In [ ]:
#GridSearchCV 를 통해 최적화된 모델들을 사용합니다.
best_models = [
    #('model_rf', RandomForestRegressor(min_samples_split=3, n_estimators=120)),
    ('model_GBR', GradientBoostingRegressor(learning_rate=0.09, loss='absolute_error', n_estimators=130)),
    #('model_ET', ExtraTreesRegressor(n_estimators=90))
]

#앙상블 기법을 위한 패키지를 불러옵니다.
from sklearn.ensemble import VotingRegressor

#앙상블 모델을 학습시켜줍니다.
voting_rg = VotingRegressor(estimators=best_models)
voting_rg.fit(x_train,y_train)

VotingRegressor(estimators=[('model_GBR',
                             GradientBoostingRegressor(learning_rate=0.09,
                                                       loss='absolute_error',
                                                       n_estimators=130))])

In [ ]:
from sklearn.metrics import *

pred = voting_rg.predict(x_test)
print("MAE:",mean_absolute_error(y_test,pred))
print("MSE:",mean_squared_error(y_test,pred))

MAE: 0.04148064888996399
MSE: 0.0042047030857190575


In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/data/dataset (1)/test_all.csv")

test_data.head(5)

,일시,연도,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,2021-01-01 00:00:00,4,11,0.244866,0.024167,0.030691,0.0,0.587,0.016
1,2021-01-01 00:00:00,4,11,0.232227,0.091944,0.012788,0.0,0.683,0.016
2,2021-01-01 01:00:00,4,11,0.222749,0.000000,0.005115,0.0,0.629,0.036
3,2021-01-01 01:00:00,4,11,0.225908,0.127222,0.012788,0.0,0.633,0.036
4,2021-01-01 02:00:00,4,11,0.206951,0.543333,0.023018,0.0,0.660,0.036


In [ ]:
test_data['일시'] = test_data['일시'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M'))

In [ ]:
test_data = test_data.set_index(["일시"])

In [ ]:
test_y = test_data["PM2.5"]
test_x = test_data.drop("PM2.5",axis =1)

test_iter = IterativeImputer()
test_x = pd.DataFrame(test_iter.fit_transform(test_x), columns=test_x.columns)

In [ ]:
test_x.set_index(test_data.index,inplace = True)
test_x.head(5)

,연도,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
일시,,,,,,,
2021-01-01 00:00:00,4.0,11.0,0.244866,0.024167,0.030691,0.0,0.587
2021-01-01 00:00:00,4.0,11.0,0.232227,0.091944,0.012788,0.0,0.683
2021-01-01 01:00:00,4.0,11.0,0.222749,0.000000,0.005115,0.0,0.629
2021-01-01 01:00:00,4.0,11.0,0.225908,0.127222,0.012788,0.0,0.633
2021-01-01 02:00:00,4.0,11.0,0.206951,0.543333,0.023018,0.0,0.660


In [ ]:
test_data

연도         0
측정소        0
기온(°C)     0
풍향(deg)    0
풍속(m/s)    0
강수량(mm)    0
습도(%)      0
PM2.5      0
dtype: int64

In [ ]:
y = pd.read_csv("/content/drive/MyDrive/data/dataset (1)/answer_sample.csv")
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78336 entries, 0 to 78335
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      78336 non-null  int64  
 1   일시      78336 non-null  object 
 2   측정소     78336 non-null  object 
 3   PM2.5   0 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


In [ ]:
le2 = LabelEncoder()
y["측정소"] = le2.fit_transform(y["측정소"])
y

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,0,NaN
1,4,01-03 01:00,0,NaN
2,4,01-03 02:00,0,NaN
3,4,01-03 03:00,0,NaN
4,4,01-03 04:00,0,NaN
...,...,...,...,...
78331,4,11-16 19:00,16,NaN
78332,4,11-16 20:00,16,NaN
78333,4,11-16 21:00,16,NaN
78334,4,11-16 22:00,16,NaN


In [ ]:
y.loc[y["연도"] == 4, "일시"] = "2021-" + y.loc[y["연도"]==4,"일시"]

In [ ]:
y['일시'] = y['일시'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M'))

In [ ]:
y = y.set_index(["일시"])

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/data/dataset (1)/answer_sample.csv.csv")

test_x = test_data.drop("id",axis=1)

for feature in cat_features:
    test_x[feature] = le.transform(test_x[feature])
    
test_x[num_features] = scaler.transform(test_x[num_features])

In [ ]:
pred = voting_rg.predict(test_x)

In [ ]:
li = []
for i in range(len(y)):
  li.append(pred[i])


In [ ]:
result = pd.read_csv("/content/drive/MyDrive/data/dataset (1)/answer_sample.csv")

result.drop(columns = ["PM2.5"],inplace = True)
result["PM2.5"] = li
result

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,0.052534
1,4,01-03 01:00,공주,0.061834
2,4,01-03 02:00,공주,0.051938
3,4,01-03 03:00,공주,0.054470
4,4,01-03 04:00,공주,0.053693
...,...,...,...,...
78331,4,11-16 19:00,홍성읍,0.085869
78332,4,11-16 20:00,홍성읍,0.087870
78333,4,11-16 21:00,홍성읍,0.091012
78334,4,11-16 22:00,홍성읍,0.074819


In [ ]:
result.to_csv("/content/drive/MyDrive/data/answer1.csv")

In [ ]:
from sklearn.model_selection import GridSearchCV

# 파라미터 선언
params = {
    'max_depth':[None,2,3,5], 
    'max_leaf_nodes':[None,3,5,7], 
    'min_samples_split':[2,4,6], 
    'min_samples_leaf':[1,2,3], 
    #'max_features':[None,'sqrt','log2',3,4,5]
    }

cv = GridSearchCV(model_r, param_grid = params, cv = 5, scoring = "mae",refit=True)

In [ ]:
print('best parameters : ', cv.best_params_)
print('best score : ', round(cv.best_score_, 4))

In [ ]:
# MSE값 도출
from sklearn.metrics import *
print("MSE :",mean_squared_error(y_test,pred))

MSE : 0.0037444781932136984
